In [1]:
import os
from glob import glob
import shutil
from pathlib import Path
import random
from tqdm import tqdm

In [2]:
indivisual_dir_path = "iteration7/Dataset_indivisual_videos/"
combined_data = "iteration7/data_combined/"

# Copy files to single directory while changing name

In [3]:
all_dir = os.listdir(indivisual_dir_path)

In [4]:
all_dir

['640-1-15',
 '640-1-12',
 '654-18',
 '651-12',
 '653-16',
 '653-18',
 '641-1-3',
 '653-15',
 '651-13',
 '651-11',
 '644-3-7',
 '651-10',
 '653-14',
 '643-1-10',
 '653-16(1)',
 '650_10',
 '644-3-4',
 '654-20',
 '643-3-1',
 '653-21',
 '640-1-6',
 '651-14',
 '654-13',
 '653-18(1)',
 '644-3-8',
 '640-1-14',
 '651-2',
 '641-1-1',
 '654-11',
 '653-19',
 '644-1-7',
 '654-10']

In [5]:
# copying all files to single directory and appending the directory name
# to the prefix
for curr_dir in all_dir:
    data_dir = os.path.join(indivisual_dir_path, curr_dir, "obj_train_data")
    print(data_dir)
    for file_name in os.listdir(data_dir):
        updated_file_name = f"{curr_dir}_{file_name}"
        src = os.path.join(data_dir, file_name)
        dst = os.path.join(combined_data, updated_file_name)
        shutil.copy2(src, dst)
    

iteration7/Dataset_indivisual_videos/640-1-15/obj_train_data
iteration7/Dataset_indivisual_videos/640-1-12/obj_train_data
iteration7/Dataset_indivisual_videos/654-18/obj_train_data
iteration7/Dataset_indivisual_videos/651-12/obj_train_data
iteration7/Dataset_indivisual_videos/653-16/obj_train_data
iteration7/Dataset_indivisual_videos/653-18/obj_train_data
iteration7/Dataset_indivisual_videos/641-1-3/obj_train_data
iteration7/Dataset_indivisual_videos/653-15/obj_train_data
iteration7/Dataset_indivisual_videos/651-13/obj_train_data
iteration7/Dataset_indivisual_videos/651-11/obj_train_data
iteration7/Dataset_indivisual_videos/644-3-7/obj_train_data
iteration7/Dataset_indivisual_videos/651-10/obj_train_data
iteration7/Dataset_indivisual_videos/653-14/obj_train_data
iteration7/Dataset_indivisual_videos/643-1-10/obj_train_data
iteration7/Dataset_indivisual_videos/653-16(1)/obj_train_data
iteration7/Dataset_indivisual_videos/650_10/obj_train_data
iteration7/Dataset_indivisual_videos/644-3-4/

# Clean dataset

In [8]:
# check for missing image or missing txt and remove them
all_files = os.listdir(combined_data)

txt_files = [f.split(".")[0] for f in all_files if f.endswith("txt")]

images = [f.split(".")[0] for f in all_files if f.endswith("PNG")]

print(len(txt_files), len(images))

missing_txt_img__file = list(set(txt_files)-((set(images))))
missing_txt_img__file

3961 3961


[]

In [7]:
# check if txt is empty then remove the txt as well as image
for file in txt_files:
    txt_file = os.path.join(combined_data, f"{file}.txt")
    data = []
    with open(txt_file,"r") as f:
        line = f.read()
        if line != '':
            data.append(line)
    if data == []:
        print(txt_file)
        img_file=os.path.join(combined_data, f"{file}.PNG")
        os.remove(txt_file)
        os.remove(img_file)

iteration7/data_combined/653-21_frame_000012.txt
iteration7/data_combined/651-14_frame_000028.txt
iteration7/data_combined/653-16_frame_000024.txt
iteration7/data_combined/651-14_frame_000112.txt
iteration7/data_combined/653-18(1)_frame_000021.txt
iteration7/data_combined/651-14_frame_000066.txt
iteration7/data_combined/651-14_frame_000393.txt
iteration7/data_combined/651-12_frame_000001.txt
iteration7/data_combined/653-15_frame_000020.txt
iteration7/data_combined/653-16_frame_000055.txt
iteration7/data_combined/651-14_frame_000062.txt
iteration7/data_combined/653-15_frame_000043.txt
iteration7/data_combined/651-14_frame_000135.txt
iteration7/data_combined/653-15_frame_000021.txt
iteration7/data_combined/651-14_frame_000421.txt
iteration7/data_combined/651-14_frame_000386.txt
iteration7/data_combined/653-16(1)_frame_000014.txt
iteration7/data_combined/654-10_frame_000001.txt
iteration7/data_combined/651-14_frame_000061.txt
iteration7/data_combined/653-15_frame_000030.txt
iteration7/dat

# Create train/val/test .txt files

In [9]:
IMG_FORMATS = 'bmp', 'dng', 'jpeg', 'jpg', 'mpo', 'png', 'tif', 'tiff', 'webp', 'pfm'  # include image suffixes
def img2label_paths(img_paths):
    # Define label paths as a function of image paths
    sa, sb = f'{os.sep}images{os.sep}', f'{os.sep}labels{os.sep}'  # /images/, /labels/ substrings
    return [sb.join(x.rsplit(sa, 1)).rsplit('.', 1)[0] + '.txt' for x in img_paths]

def autosplit(path, weights=(0.8, 0.1, 0.1), annotated_only=False):
    """ Autosplit a dataset into train/val/test splits and save path/autosplit_*.txt files
    Usage: from utils.dataloaders import *; autosplit()
    Arguments
        path:            Path to images directory
        weights:         Train, val, test weights (list, tuple)
        annotated_only:  Only use images with an annotated txt file
    """
    path = Path(path)  # images dir
    files = sorted(x for x in path.rglob('*.*') if x.suffix[1:].lower() in IMG_FORMATS)  # image files only
    n = len(files)  # number of files
    random.seed(0)  # for reproducibility
    indices = random.choices([0, 1, 2], weights=weights, k=n)  # assign each image to a split

    txt = ['autosplit_train.txt', 'autosplit_val.txt', 'autosplit_test.txt']  # 3 txt files
    for x in txt:
        if (path.parent / x).exists():
            (path.parent / x).unlink()  # remove existing

    print(f'Autosplitting images from {path}' + ', using *.txt labeled images only' * annotated_only)
    for i, img in tqdm(zip(indices, files), total=n):
        if not annotated_only or Path(img2label_paths([str(img)])[0]).exists():  # check label
            with open(path.parent / txt[i], 'a') as f:
                f.write(f'./{img.relative_to(path.parent).as_posix()}' + '\n')  # add image to txt file


In [10]:
autosplit(combined_data)

Autosplitting images from iteration7/data_combined


100%|████████████████████████████████████| 3961/3961 [00:00<00:00, 53241.33it/s]
